# Open AI

[![Index](https://img.shields.io/badge/Index-blue)](../index.ipynb)
[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/digillia/Digillia-Colab/blob/main/tools/openai.ipynb)

In [ ]:
# Supprimer les commentaires pour installer
# !pip3 install -q -U openai